### **Descarga e importe de bibliotecas**

In [1]:
%%capture
# Instalar y actualizar bibliotecas necesarias
!pip install roboflow
!pip install ultralytics

In [2]:
# Importar bibliotecas a usar y deshabilitar WanDB
import os
import yaml
import wandb
import shutil
import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from roboflow import Roboflow
from ultralytics import YOLO


os.environ['WANDB_DISABLED'] = 'true'

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### **Descarga y acondicionamiento de la base de datos de Mamitas**

In [3]:
# Descargar repositorio de aisladores
rf = Roboflow(api_key="9smAmQgaD8pNOTsDMYKR")
project = rf.workspace("mamitas").project("mamitas-yolo")
version = project.version(3)
dataset = version.download("yolov11")
!mkdir datasets
!mv ./MAMITAS-YOLO-3 ./datasets/

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to MAMITAS-YOLO-3 in yolov11:: 100%|██████████| 844/844 [00:00<00:00, 9469.75it/s]


In [4]:
# Creación del directorio de las partes de la base de datos
data = {
    'path': './MAMITAS-YOLO-3',
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['feet'],
    'roboflow': {
        'license': 'CC BY 4.0',
        'project': 'MAMITAS-YOLO',
        'url': 'https://universe.roboflow.com/mamitas/mamitas-yolo/dataset/3',
        'version': '3',
        'workspace': 'Mamitas'}
}

# Especificar el directorio del archivo
file_path = './datasets/MAMITAS-YOLO-3/data.yaml'

# Escribir la información sobre el archivo YAML
with open(file_path, 'w') as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False)

### **Entrenamiento del modelo YOLOv11**

### **Funcionamiento del modelo YOLO (You Only Look Once)**

YOLO para segmentación de objetos es una extensión del modelo YOLO de detección, diseñada para identificar la posición y forma precisa de los objetos en una imagen en tiempo real. En lugar de solo predecir bounding boxes, este modelo también genera máscaras de segmentación para cada objeto detectado. Funciona dividiendo la imagen en una cuadrícula, donde cada celda predice la presencia de un objeto, sus coordenadas, la clase y la máscara correspondiente.

La función de pérdida total en un modelo YOLO de segmentación se compone de varios términos que permiten entrenar simultáneamente la detección y la segmentación de objetos. A continuación se muestran las fórmulas principales:

1. **Pérdida de Localización:**  
   Mide el error en las coordenadas de las cajas delimitadoras.
   $$ 
   L_{\text{loc}} = \sum_{i \in \text{objetos}} \left[(b_x - \hat{b}_x)^2 + (b_y - \hat{b}_y)^2 + (b_w - \hat{b}_w)^2 + (b_h - \hat{b}_h)^2 \right]
   $$

2. **Pérdida de Confianza:**  
   Penaliza la diferencia entre la puntuación de confianza predicha \( c \) y la presencia real del objeto \( \hat{c} \).
   $$ 
   L_{\text{conf}} = \sum_{i} (c - \hat{c})^2
   $$

3. **Pérdida de Clasificación:**  
   Utiliza la entropía cruzada para medir el error en la predicción de la clase.
   $$ 
   L_{\text{class}} = - \sum_{i \in \text{objetos}} \sum_{j=1}^{C} \hat{p}_j \log(p_j)
   $$

4. **Pérdida de Segmentación:**  
   Compara la máscara predicha \( M \) con la máscara real \( \hat{M} \) usando la entropía cruzada pixel a pixel.
   $$ 
   L_{\text{seg}} = - \sum_{pixels} \left[\hat{M}(pixel) \log(M(pixel)) + (1-\hat{M}(pixel)) \log(1-M(pixel))\right]
   $$

La función de pérdida total es una combinación ponderada de estos términos:
$$ 
L_{\text{total}} = \lambda_{\text{loc}} L_{\text{loc}} + \lambda_{\text{conf}} L_{\text{conf}} + \lambda_{\text{class}} L_{\text{class}} + \lambda_{\text{seg}} L_{\text{seg}}
$$

donde $$(\lambda_{\text{loc}}, \lambda_{\text{conf}}, \lambda_{\text{class}},\lambda_{\text{seg}})$$ son coeficientes que balancean la importancia de cada componente.


In [ ]:
# Entrenar YOLOv11 para detección de Mamitas

# Carga de modelo preentrenado
model = YOLO('yolo11m-seg.pt')

# Entrenar el modelo
results = model.train(data='./datasets/MAMITAS-YOLO-3/data.yaml', epochs=100, imgsz=512, device=[0,1])

100%|██████████| 43.3M/43.3M [00:01<00:00, 43.1MB/s]


Ultralytics 8.3.99 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=./datasets/MAMITAS-YOLO-3/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

100%|██████████| 755k/755k [00:00<00:00, 32.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

### **Rendimiento del modelo sobre datos de validación**

In [6]:
# Validar el modelo entrenado

# Carga de modelo entrenado
model = YOLO('./runs/segment/train/weights/best.pt',task='segment') 

In [7]:
# Realizar validación
validation_results = model.val(data='./datasets/MAMITAS-YOLO-3/data.yaml',
                               imgsz=512,
                               batch=16,
                               conf=0.8,
                               iou=0.6,
                               device='0')

Ultralytics 8.3.99 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m-seg summary (fused): 138 layers, 22,336,083 parameters, 0 gradients, 123.0 GFLOPs


val: Scanning /kaggle/working/datasets/MAMITAS-YOLO-3/valid/labels.cache... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.56it/s]


                   all         37         74          1          1      0.995      0.995          1          1      0.995      0.995


invalid value encountered in less
invalid value encountered in less


Speed: 0.3ms preprocess, 28.7ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to runs/segment/val


In [8]:
# Cargar la imagen
image = mpimg.imread("./runs/segment/val/confusion_matrix.png")
image_1 = mpimg.imread("./runs/segment/val/MaskPR_curve.png")

# Mostrar la matriz de confusión imagen
fig, axis = plt.subplots(1,2,figsize=(18,8))
axis[0].imshow(image)
axis[0].axis('off')
axis[1].imshow(image_1)
axis[1].axis('off')
plt.show()